In [19]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [20]:
import folium
from folium.plugins import MarkerCluster
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
import requests
from folium.plugins import MiniMap

In [21]:
roadkill_2019 = pd.read_csv(r'G:\내 드라이브\공모전1\데이터\로드킬\한국도로공사_로드킬 데이터 정보_2019.csv', encoding = 'cp949')
roadkill_2020 = pd.read_csv(r'G:\내 드라이브\공모전1\데이터\로드킬\한국도로공사_로드킬 데이터 정보_2020.csv', encoding = 'cp949')
roadkill_2021 = pd.read_csv(r'G:\내 드라이브\공모전1\데이터\로드킬\한국도로공사_로드킬 데이터 정보_2021.csv', encoding = 'cp949')
roadkill_2022 = pd.read_csv(r'G:\내 드라이브\공모전1\데이터\로드킬\한국도로공사_로드킬 데이터 정보_2022.csv', encoding = 'cp949')
roadkill_df = pd.read_csv(r'G:\내 드라이브\공모전1\전처리\한국도로공사_로드킬 데이터 정보_2022~2019 .csv', encoding = 'cp949')

In [22]:
route = pd.read_csv(r'G:\내 드라이브\공모전1\전처리\생태통로_pre.csv')
fence = pd.read_csv(r'G:\내 드라이브\공모전1\데이터\유도울타리.csv', encoding = 'cp949')
animal = pd.read_csv(r'G:\내 드라이브\공모전1\데이터\국내 포유류 서식 분포_pre.csv', encoding = 'cp949')

In [23]:
# 필요없는 컬럼 드랍
columns_to_drop = ['wkt_geom', 'SPCE_ID', 'EXAMIN_REA', 'SPCS_SCNCE', 'EXAMIN_YEA', 'EXAMIN_BEG', 'EXAMIN_END']
animal_df = animal.drop(columns=columns_to_drop, axis=1)

# 'longitude'와 'latitude'가 무한대인 행 제거
animal_df = animal_df[~animal_df['longitude'].isin([np.inf, -np.inf])]
animal_df = animal_df[~animal_df['latitude'].isin([np.inf, -np.inf])]

# latitude, longitude 컬럼명을 한국어로 변경
animal_df = animal_df.rename(columns={'latitude': '위도', 'longitude': '경도'})

# 삭제하고자 하는 값들 / 조류, 설치류 제거
values_to_remove = ['큰수염박쥐', '북방애기박쥐', '비단털쥐', '관코박쥐', '제주땃쥐', '우수리땃쥐', 
                    '집박쥐', '양박쥐', '갈밭쥐', '등줄쥐', '멧밭쥐', '우수리박쥐', '큰집박쥐', 
                    '쇠윗수염박쥐', '큰발윗수염박쥐', '집쥐', '흰넓적다리붉은쥐', '작은땃쥐', '뒤쥐', 
                    '생쥐', '관박쥐', '안주애기박쥐', '대륙밭쥐', '문둥이박쥐', '흰배윗수염박쥐', 
                    '대륙쇠큰수염박쥐', '돌고래', '다람쥐', '청설모', '고슴도치', '뉴트리아']

# 'SPCS_LCNM'이 삭제하고자 하는 값들을 포함하지 않는 행 선택
animal_df = animal_df[~animal_df['SPCS_LCNM'].isin(values_to_remove)]

In [24]:
# 필요없는 컬럼 드랍
columns_to_drop2 = ['번호', '생태통로명', '주소', '차선_수', '관리기관', '생태통로유형', '설치기관']
route_df = route.drop(columns=columns_to_drop2, axis=1)

In [25]:
# roadkill_df를 result_df에 복사합니다.
result_df = roadkill_df.copy()

In [26]:
# result_df에 울타리 평균 길이와 울타리 평균 높이를 저장할 열 추가
result_df['울타리 평균 길이'] = 0
result_df['울타리 평균 높이'] = 0

# 각 5KM 데이터에 대해 fence에서 범위에 해당하는 row를 찾고 연장과 높이의 평균을 계산하여 result_df에 저장
for index, row in result_df.iterrows():
    start_point = row[' 5km ']
    end_point = row[' 5km ']
    
    # 설치시점과 설치종점 범위 내의 데이터 추출
    relevant_fences = fence[(fence['설치시점'] <= start_point) & (fence['설치종점'] >= end_point)]
    
    # 연장과 높이의 평균 계산
    avg_extension = relevant_fences['연장'].mean()
    avg_height = relevant_fences['높이'].mean()
    
    # 결과 데이터프레임에 저장
    result_df.at[index, '울타리 평균 길이'] = avg_extension
    result_df.at[index, '울타리 평균 높이'] = avg_height


In [27]:
#동물	생활 활동 반경 (국내)
#고라니	약 1~10 km²
#두더지	약 수 미터에서 수십 미터
#멧돼지	약 5~20 km²
#너구리	약 0.5~5 km²
#오소리	약 2~10 km²
#노루      약 1~10 km²
#말사슴      약 1~10 km²
#족제비	약 0.5~5 km²
#멧토끼	약 0.1~0.5 km²

In [28]:
# 동물들의 활동 반경 딕셔너리로 정의
activity_range = {
    '고라니': 5,
    '멧돼지': 10,
    '족제비': 2.5,
    '두더지': 0.25,
    '오소리': 5,
    '너구리': 2.5,
    '노루': 5,
    '말사슴': 5,
    '멧토끼': 0.25
}

# 각 동물의 평균 활동 반경 계산
mean_activity_range = animal_df['SPCS_LCNM'].apply(lambda x: activity_range[x]).mean()

In [29]:
mean_activity_range

4.461890243902439

In [30]:
from geopy.distance import geodesic

In [31]:
def count_eco_trails_within_radius(lat, lon, eco_trails_df, radius_km=5):
    
    #반경 내 생태통로 수를 계산
    
    count = 0
    for idx, trail in eco_trails_df.iterrows():
        trail_location = (trail['위도'], trail['경도'])
        distance = geodesic((lat, lon), trail_location).kilometers
        if distance <= radius_km:
            count += 1
    return count

# result_df에 '반경 생태통로 수'라는 새로운 열을 만들고 값을 계산하여 채웁니다.
result_df['반경 생태통로 수'] = result_df.apply(
    lambda row: count_eco_trails_within_radius(row['위도'], row['경도'], route, radius_km = mean_activity_range),
    axis=1
)

In [32]:
def count_animals_within_radius(lat, lon, animal_df, activity_range):
    
    #반경 내 동물 개체 수를 계산
    
    count_dict = {animal: 0 for animal in activity_range.keys()}
    for idx, animal in animal_df.iterrows():
        animal_location = (animal['위도'], animal['경도'])
        for animal_type, range_value in activity_range.items():
            distance = geodesic((lat, lon), animal_location).kilometers
            if distance <= range_value:
                count_dict[animal['SPCS_LCNM']] += 1
    return count_dict

# result_df에 '반경 동물 개체 수'라는 새로운 열을 만들고 값을 계산
result_df['반경 동물 개체 수'] = result_df.apply(
    lambda row: count_animals_within_radius(row['위도'], row['경도'], animal_df, activity_range),
    axis=1
)

In [33]:
result_df.head()

,본부명,지사명,노선명,구간,방 향,5km,발생건수,위도,경도,발생년도,울타리 평균 길이,울타리 평균 높이,반경 생태통로 수,반경 동물 개체 수
0,강원,홍천,중앙선,315~320,부산,315,13,37.340987,127.912591,2022,351.900000,1.450000,0,"{'고라니': 113, '멧돼지': 11, '족제비': 15, '두더지': 75, ..."
1,강원,홍천,중앙선,315~320,춘천,315,7,37.340987,127.912591,2022,351.900000,1.450000,0,"{'고라니': 113, '멧돼지': 11, '족제비': 15, '두더지': 75, ..."
2,강원,홍천,중앙선,385~390,춘천,385,6,37.838769,127.777996,2022,279.000000,1.625000,2,"{'고라니': 96, '멧돼지': 35, '족제비': 16, '두더지': 36, '..."
3,강원,강릉,동해선,65~70,속초,65,4,37.802916,128.823887,2022,284.420455,1.715909,0,"{'고라니': 74, '멧돼지': 38, '족제비': 2, '두더지': 20, '오..."
4,강원,대관령,영동선,230~235,인천,230,4,37.754931,128.796506,2022,277.187500,1.562500,2,"{'고라니': 72, '멧돼지': 43, '족제비': 1, '두더지': 8, '오소..."


In [34]:
result_df.head()

,본부명,지사명,노선명,구간,방 향,5km,발생건수,위도,경도,발생년도,울타리 평균 길이,울타리 평균 높이,반경 생태통로 수,반경 동물 개체 수
0,강원,홍천,중앙선,315~320,부산,315,13,37.340987,127.912591,2022,351.900000,1.450000,0,"{'고라니': 113, '멧돼지': 11, '족제비': 15, '두더지': 75, ..."
1,강원,홍천,중앙선,315~320,춘천,315,7,37.340987,127.912591,2022,351.900000,1.450000,0,"{'고라니': 113, '멧돼지': 11, '족제비': 15, '두더지': 75, ..."
2,강원,홍천,중앙선,385~390,춘천,385,6,37.838769,127.777996,2022,279.000000,1.625000,2,"{'고라니': 96, '멧돼지': 35, '족제비': 16, '두더지': 36, '..."
3,강원,강릉,동해선,65~70,속초,65,4,37.802916,128.823887,2022,284.420455,1.715909,0,"{'고라니': 74, '멧돼지': 38, '족제비': 2, '두더지': 20, '오..."
4,강원,대관령,영동선,230~235,인천,230,4,37.754931,128.796506,2022,277.187500,1.562500,2,"{'고라니': 72, '멧돼지': 43, '족제비': 1, '두더지': 8, '오소..."
